<a href="https://colab.research.google.com/github/yo20mom/CUK_Learning/blob/master/KerasDeepLearning/fashion_mnist_%E1%84%86%E1%85%A1%E1%84%89%E1%85%A1%E1%84%8D%E1%85%A1%E1%86%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, utils, callbacks, datasets, layers, applications, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist, fashion_mnist
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Activation, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras import backend as K

import matplotlib.pyplot as plt
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


# 이미지 생성을 위한 generator 추가

In [3]:
image_generator = ImageDataGenerator(
    rotation_range=.13,
    zoom_range=0.2,
    shear_range=0.6,
    # rescale=1. / 255.,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    vertical_flip=False,
   # validation_split=0.2
    )

In [4]:
def run_gen(gen, x):
  augment_size = len(x)
  return gen.flow(x, np.zeros(augment_size), batch_size=augment_size, shuffle=False).next()[0]

def extract(x, y, target):
  t = np.where(y == target)
  x_result = x[t].copy()
  y_result = y[t].copy()
  print(len(x_result), len(y_result))
  return x_result, y_result

def augments(gen, np_img, np_answer):
  x_top, y_top = extract(np_img, np_answer, 0)
  x_shirt, y_shirt = extract(np_img, np_answer, 6)
  x_pullover, y_pullover = extract(np_img, np_answer, 2)
  x_coat, y_coat = extract(np_img, np_answer, 4)
  x_shirt2, y_shirt2 = extract(np_img, np_answer, 6)
  
  print("정상 1")
  tops = run_gen(gen, x_top)
  shirts = run_gen(gen, x_shirt)
  pullover = run_gen(gen, x_pullover)
  coats = run_gen(gen, x_coat)  
  shirts2 = run_gen(gen, x_shirt2)

  print("정상 2")
  np_img = np.concatenate((np_img, tops))
  np_img = np.concatenate((np_img, shirts))
  np_img = np.concatenate((np_img, pullover))
  np_img = np.concatenate((np_img, coats))
  np_img = np.concatenate((np_img, shirts2))

  print("x_train size : ", len(np_img))
  print("정상 3")
  np_answer = np.concatenate((np_answer, y_top))
  np_answer = np.concatenate((np_answer, y_shirt))
  np_answer = np.concatenate((np_answer, y_pullover))
  np_answer = np.concatenate((np_answer, y_coat))
  np_answer = np.concatenate((np_answer, y_shirt2))
  
  print("y_train size : ", len(np_answer))
  return np_img.copy(), np_answer.copy()

In [5]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [6]:
x_train, y_train = augments(image_generator, x_train, y_train)
print(x_train.shape, y_train.shape)

6000 6000
6000 6000
6000 6000
6000 6000
6000 6000
정상 1
정상 2
x_train size :  90000
정상 3
y_train size :  90000
(90000, 28, 28, 1) (90000,)


# 이미지 generator 실행함수

하나의 row마다 augment size만큼 증가 시킨다.



In [7]:
augment_size = len(np.where(y_train == 6)[0])
x_augmented = x_train[np.where(y_train == 6)].copy()
y_augmented = y_train[np.where(y_train == 6)].copy()

x_augmented = image_generator.flow(x_augmented, 
                                   np.zeros(augment_size), 
                                   batch_size=augment_size, 
                                   shuffle=False).next()[0]

x_train = np.concatenate((x_train, x_augmented))
y_train = np.concatenate((y_train, y_augmented))


In [8]:
augment_size = len(np.where(y_train == 6)[0])
x_augmented = x_train[np.where(y_train == 6)].copy()
y_augmented = y_train[np.where(y_train == 6)].copy()

x_augmented = image_generator.flow(x_augmented, 
                                   np.zeros(augment_size), 
                                   batch_size=augment_size, 
                                   shuffle=False).next()[0]

x_train = np.concatenate((x_train, x_augmented))
y_train = np.concatenate((y_train, y_augmented))

print(x_train.shape)
print(y_train.shape)

(144000, 28, 28, 1)
(144000,)


In [9]:
augment_size = len(np.where(y_train == 0)[0])
x_augmented = x_train[np.where(y_train == 0)].copy()
y_augmented = y_train[np.where(y_train == 0)].copy()

x_augmented = image_generator.flow(x_augmented, 
                                   np.zeros(augment_size), 
                                   batch_size=augment_size, 
                                   shuffle=False).next()[0]

x_train = np.concatenate((x_train, x_augmented))
y_train = np.concatenate((y_train, y_augmented))

print(x_train.shape)
print(y_train.shape)

(156000, 28, 28, 1)
(156000,)


In [10]:
augment_size = 500000

random_mask = np.random.randint(x_train.shape[0], size= augment_size)
x_augmented = x_train[random_mask].copy()
y_augmented = y_train[random_mask].copy()

In [11]:
x_augmented = image_generator.flow(x_augmented, 
                                   np.zeros(augment_size), 
                                   batch_size=augment_size, 
                                   shuffle=False).next()[0]

x_train = np.concatenate((x_train, x_augmented))
y_train = np.concatenate((y_train, y_augmented))

print(x_train.shape)
print(y_train.shape)

(656000, 28, 28, 1)
(656000,)


In [12]:
#x_train, y_train = augments(image_generator, x_train, y_train)

In [13]:
x_train.shape, y_train.shape

((656000, 28, 28, 1), (656000,))

In [14]:
def my_model(filters=[32, 64], kernel_size=(3, 3), pool_size=(2, 2), act='relu'):
  width, height, depth, classes = 28, 28, 1, 10
  
  model = Sequential()
  input_shape = (height, width, depth)
  chan_dim = -1

  for i, x in enumerate(filters):
    if i == 0:
      model.add(Conv2D(filters=x, 
                       kernel_size=kernel_size, 
                       padding='same',
                       input_shape=input_shape))
    else:
      model.add(Conv2D(filters=x, kernel_size=kernel_size, padding='same'))

    model.add(BatchNormalization(axis=chan_dim))
    model.add(Activation(act))
    
    model.add(Conv2D(filters=x, kernel_size=kernel_size, padding='same'))
    model.add(BatchNormalization(axis=chan_dim))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.3))

#   model.add(tf.keras.layers.GlobalMaxPool2D())
  model.add(Flatten())
  
  model.add(Dense(512))
  model.add(BatchNormalization())
  model.add(Activation(act))
  model.add(Dropout(0.4))

  model.add(Dense(128))
  model.add(BatchNormalization())
  model.add(Activation(act))
  model.add(Dropout(0.4))

  model.add(Dense(classes))
  model.add(Activation('softmax'))

  return model

In [15]:
x_train.shape, x_test.shape

((656000, 28, 28, 1), (10000, 28, 28, 1))

#값 정규화
ㅇ이미지 데이터 generator에서 작업하기 때문에 주석처리

In [16]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# 원핫 인코딩으로 변경

In [17]:
classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes=classes)
y_test = to_categorical(y_test, num_classes=classes)

In [18]:
# 모델 저장 기준이 되는 콜백 import
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [19]:
# 체크할 포인트
check_point = ModelCheckpoint('best_model.h5', 
                              monitor='val_accuracy',
                              verbose=1,
                              save_best_only=True)

# 이른 스탑을 위한 클래스
early_stopping = EarlyStopping(monitor='val_accuracy',
                               min_delta=0,
                               patience=7,
                               verbose=1)

# 모델 컴파일 및 학습


In [20]:
bat_size = 128
epochs = 33
lr = 0.001
bs = 128
ep = 50


optimizer = Adam(learning_rate=lr, 
                #decay=lr / ep, 
                beta_1=0.9,
                beta_2=0.999)


model = my_model()

model.compile(optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy'])

history = model.fit(x_train,
                    y_train,
                    batch_size=bs,
                    validation_split=0.3,
                    validation_data=(x_test, y_test),
                    epochs=ep,
                    verbose=2,
                    callbacks=[early_stopping, check_point])

Epoch 1/50
3588/3588 - 90s - loss: 0.5976 - accuracy: 0.7752 - val_loss: 0.4840 - val_accuracy: 0.8090

Epoch 00001: val_accuracy improved from -inf to 0.80897, saving model to best_model.h5
Epoch 2/50
3588/3588 - 43s - loss: 0.4246 - accuracy: 0.8408 - val_loss: 0.3883 - val_accuracy: 0.8508

Epoch 00002: val_accuracy improved from 0.80897 to 0.85077, saving model to best_model.h5
Epoch 3/50
3588/3588 - 47s - loss: 0.3789 - accuracy: 0.8583 - val_loss: 0.3777 - val_accuracy: 0.8613

Epoch 00003: val_accuracy improved from 0.85077 to 0.86129, saving model to best_model.h5
Epoch 4/50
3588/3588 - 44s - loss: 0.3520 - accuracy: 0.8693 - val_loss: 0.3084 - val_accuracy: 0.8819

Epoch 00004: val_accuracy improved from 0.86129 to 0.88187, saving model to best_model.h5
Epoch 5/50
3588/3588 - 44s - loss: 0.3306 - accuracy: 0.8773 - val_loss: 0.3122 - val_accuracy: 0.8800

Epoch 00005: val_accuracy did not improve from 0.88187
Epoch 6/50
3588/3588 - 44s - loss: 0.3150 - accuracy: 0.8826 - val_l

In [21]:
loaded_model = load_model('best_model.h5')

prob_pred = loaded_model.predict(x_test)
pred = loaded_model.evaluate(x_test, y_test)
print(pred)

prob_label = prob_pred.argmax(axis=-1)

np.savetxt('y_pred.csv', prob_label, fmt='%d')

313/313 [==============================] - 1s 3ms/step - loss: 0.1661 - accuracy: 0.9473
[0.16612492501735687, 0.9473000168800354]


In [22]:
# show a nicely formatted classification report
from sklearn.metrics import classification_report

label_names = ["top", "trouser", "pullover", "dress", "coat", "sandal", "shirt", "sneaker", "bag", "ankle boot"]

print("[INFO] evaluating network...")
print(classification_report(y_test.argmax(axis=1), prob_pred.argmax(axis=1), target_names=label_names))

[INFO] evaluating network...
              precision    recall  f1-score   support

         top       0.92      0.88      0.90      1000
     trouser       1.00      0.99      0.99      1000
    pullover       0.95      0.92      0.93      1000
       dress       0.94      0.95      0.95      1000
        coat       0.91      0.94      0.93      1000
      sandal       0.99      0.98      0.99      1000
       shirt       0.83      0.86      0.84      1000
     sneaker       0.96      0.98      0.97      1000
         bag       0.99      1.00      0.99      1000
  ankle boot       0.98      0.97      0.97      1000

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000

